<a href="https://colab.research.google.com/github/Indu-Nagraj/Portfolio/blob/main/Copy_of_Neural_Machine_Translation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install chart-studio

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.4/64.4 kB 3.9 MB/s eta 0:00:00


In [ ]:
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow as tf

#tf.enable_eager_execution()

import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

import unicodedata
import re
import numpy as np
import time
import string

import chart_studio.plotly
import chart_studio.plotly as py
from plotly.offline import init_notebook_mode, iplot
#%plotly.offline.init_notebook_mode(connected=True)
import plotly.graph_objs as go

import os


file = '/content/NMT dataset 1990.txt' # the path according to our system
lines = open(file, encoding='UTF-8').read().strip().split('\n')
lines[5000:5010]

print("total number of records: ",len(lines))

total number of records:  1990


In [ ]:
punc = set(string.punctuation) # Set of all special characters
digits = str.maketrans('', '', string.digits)

In [ ]:
def preprocess_eng_sentence(sent):
    '''Function to preprocess English sentence'''
    sent = sent.lower() # lower casing
    sent = re.sub("'", '', sent) # remove the quotation marks if any
    sent = ''.join(ch for ch in sent if ch not in punc)
    sent = sent.translate(digits) # remove the digits
    sent = sent.strip()
    sent = re.sub(" +", " ", sent) # remove extra spaces
    sent = '<start> ' + sent + ' <end>' # add <start> and <end> tokens
    return sent

In [ ]:
def preprocess_kan_sentence(sent):
    '''Function to preprocess Portuguese sentence'''
    sent = re.sub("'", '', sent) # remove the quotation marks if any
    sent = ''.join(ch for ch in sent if ch not in punc)
    sent = sent.strip()
    sent = re.sub(" +", " ", sent) # remove extra spaces
    sent = '<start> ' + sent + ' <end>' # add <start> and <end> tokens
    return sent

In [ ]:
# Generate pairs of cleaned English and Portuguese sentences
sent_pairs = []
for line in lines:
    sent_pair = []
    eng = line.rstrip().split('\t')[0]
    kan = line.rstrip().split('\t')[1]
    eng = preprocess_eng_sentence(eng)
    sent_pair.append(eng)
    kan = preprocess_kan_sentence(kan)
    sent_pair.append(kan)
    sent_pairs.append(sent_pair)
sent_pairs[5000:5010]

[]

In [ ]:
# This class creates a word -> index mapping (e.g,. "dad" -> 5) and vice-versa
# (e.g., 5 -> "dad") for each language,
class Indexing():
    def __init__(self, lang):
        self.lang = lang
        self.word2idx = {}
        self.idx2word = {}
        self.vocab = set()

        self.create_index()

    def create_index(self):
        for phrase in self.lang:
            self.vocab.update(phrase.split(' '))

        self.vocab = sorted(self.vocab)

        self.word2idx['<pad>'] = 0
        for index, word in enumerate(self.vocab):
            self.word2idx[word] = index + 1

        for word, index in self.word2idx.items():
            self.idx2word[index] = word

In [ ]:
def max_length(tensor):
    return max(len(t) for t in tensor)

In [ ]:
def load_dataset(pairs, num_examples):
    # pairs => already created cleaned input, output pairs

    # index language using the class defined above
    inp_lang = Indexing(en for en, ka in pairs)
    targ_lang = Indexing(ka for en, ka in pairs)

    # Vectorize the input and target languages

    # English sentences
    input_tensor = [[inp_lang.word2idx[s] for s in en.split(' ')] for en, ka in pairs]

    # Marathi sentences
    target_tensor = [[targ_lang.word2idx[s] for s in ka.split(' ')] for en, ka in pairs]

    # Calculate max_length of input and output tensor
    # Here, we'll set those to the longest sentence in the dataset
    max_length_inp, max_length_tar = max_length(input_tensor), max_length(target_tensor)

    # Padding the input and output tensor to the maximum length
    input_tensor = tf.keras.preprocessing.sequence.pad_sequences(input_tensor,
                                                                 maxlen=max_length_inp,
                                                                 padding='post')

    target_tensor = tf.keras.preprocessing.sequence.pad_sequences(target_tensor,
                                                                  maxlen=max_length_tar,
                                                                  padding='post')

    return input_tensor, target_tensor, inp_lang, targ_lang, max_length_inp, max_length_tar


In [ ]:
input_tensor, target_tensor, inp_lang, targ_lang, max_length_inp, max_length_targ = load_dataset(sent_pairs, len(lines))

In [ ]:
# Creating training and validation sets using an 80-20 split
input_tensor_train, input_tensor_val, target_tensor_train, target_tensor_val = train_test_split(input_tensor, target_tensor, test_size=0.1, random_state = 101)

# Show length
len(input_tensor_train), len(target_tensor_train), len(input_tensor_val), len(target_tensor_val)

(1791, 1791, 199, 199)

In [ ]:
BUFFER_SIZE = len(input_tensor_train)
BATCH_SIZE = 64
N_BATCH = BUFFER_SIZE//BATCH_SIZE
embedding_dim = 256
units = 1024
vocab_inp_size = len(inp_lang.word2idx)
vocab_tar_size = len(targ_lang.word2idx)

dataset = tf.data.Dataset.from_tensor_slices((input_tensor_train, target_tensor_train)).shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)

In [ ]:
def gru(units):

    return tf.keras.layers.GRU(units,
                                   return_sequences=True,
                                   return_state=True,
                                   recurrent_activation='sigmoid',
                                   recurrent_initializer='glorot_uniform')

In [ ]:
class Encoder(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, enc_units, batch_sz):
        super(Encoder, self).__init__()
        self.batch_sz = batch_sz
        self.enc_units = enc_units
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        self.gru = gru(self.enc_units)

    def call(self, x, hidden):
        x = self.embedding(x)
        output, state = self.gru(x, initial_state = hidden)
        return output, state

    def initialize_hidden_state(self):
        return tf.zeros((self.batch_sz, self.enc_units))


In [ ]:
class Decoder(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, dec_units, batch_sz):
        super(Decoder, self).__init__()
        self.batch_sz = batch_sz
        self.dec_units = dec_units
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        self.gru = gru(self.dec_units)
        self.fc = tf.keras.layers.Dense(vocab_size)

        # used for attention
        self.W1 = tf.keras.layers.Dense(self.dec_units)
        self.W2 = tf.keras.layers.Dense(self.dec_units)
        self.V = tf.keras.layers.Dense(1)

    def call(self, x, hidden, enc_output):

        hidden_with_time_axis = tf.expand_dims(hidden, 1)

        # score shape == (batch_size, max_length, 1)
        # we get 1 at the last axis because we are applying tanh(FC(EO) + FC(H)) to self.V
        score = self.V(tf.nn.tanh(self.W1(enc_output) + self.W2(hidden_with_time_axis)))

        # attention_weights shape == (batch_size, max_length, 1)
        attention_weights = tf.nn.softmax(score, axis=1)

        # context_vector shape after sum == (batch_size, hidden_size)
        context_vector = attention_weights * enc_output
        context_vector = tf.reduce_sum(context_vector, axis=1)

        # x shape after passing through embedding == (batch_size, 1, embedding_dim)
        x = self.embedding(x)

        # x shape after concatenation == (batch_size, 1, embedding_dim + hidden_size)
        x = tf.concat([tf.expand_dims(context_vector, 1), x], axis=-1)

        # passing the concatenated vector to the GRU
        output, state = self.gru(x)

        # output shape == (batch_size * 1, hidden_size)
        output = tf.reshape(output, (-1, output.shape[2]))

        # output shape == (batch_size * 1, vocab)
        x = self.fc(output)

        return x, state, attention_weights

    def initialize_hidden_state(self):
        return tf.zeros((self.batch_sz, self.dec_units))

In [ ]:
encoder = Encoder(vocab_inp_size, embedding_dim, units, BATCH_SIZE)
decoder = Decoder(vocab_tar_size, embedding_dim, units, BATCH_SIZE)

In [ ]:
optimizer = tf.optimizers.Adam()

def loss_function(real, pred):
    mask = 1 - np.equal(real, 0)
    loss_ = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=real, logits=pred) * mask
    return tf.reduce_mean(loss_)

In [ ]:
checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
checkpoint = tf.train.Checkpoint(optimizer=optimizer,
                                 encoder=encoder,
                                 decoder=decoder)

In [ ]:
EPOCHS = 40

for epoch in range(EPOCHS):
    start = time.time()

    hidden = encoder.initialize_hidden_state()
    total_loss = 0

    for (batch, (inp, targ)) in enumerate(dataset):
        loss = 0

        with tf.GradientTape() as tape:
            enc_output, enc_hidden = encoder(inp, hidden)

            dec_hidden = enc_hidden

            dec_input = tf.expand_dims([targ_lang.word2idx['<start>']] * BATCH_SIZE, 1)

            # Teacher forcing - feeding the target as the next input
            for t in range(1, targ.shape[1]):
                # passing enc_output to the decoder
                predictions, dec_hidden, _ = decoder(dec_input, dec_hidden, enc_output)

                loss += loss_function(targ[:, t], predictions)

                # using teacher forcing
                dec_input = tf.expand_dims(targ[:, t], 1)

        batch_loss = (loss / int(targ.shape[1]))

        total_loss += batch_loss

        variables = encoder.variables + decoder.variables

        gradients = tape.gradient(loss, variables)

        optimizer.apply_gradients(zip(gradients, variables))

        if batch % 100 == 0:
            print('Epoch {} Batch {} Loss {:.4f}'.format(epoch + 1,
                                                         batch,
                                                         batch_loss.numpy()))
    # saving (checkpoint) the model every epoch
    checkpoint.save(file_prefix = checkpoint_prefix)


    print('Time taken for 1 epoch {} sec\n'.format(time.time() - start))

Epoch 1 Batch 0 Loss 3.2948
Time taken for 1 epoch 313.0074305534363 sec

Epoch 2 Batch 0 Loss 3.1411
Time taken for 1 epoch 323.46373462677 sec

Epoch 3 Batch 0 Loss 2.7510
Time taken for 1 epoch 324.6709449291229 sec

Epoch 4 Batch 0 Loss 2.5933
Time taken for 1 epoch 314.43249702453613 sec

Epoch 5 Batch 0 Loss 2.0832
Time taken for 1 epoch 320.90531158447266 sec

Epoch 6 Batch 0 Loss 1.8233
Time taken for 1 epoch 325.19939851760864 sec

Epoch 7 Batch 0 Loss 1.3010
Time taken for 1 epoch 325.09987115859985 sec

Epoch 8 Batch 0 Loss 1.1004
Time taken for 1 epoch 323.4932861328125 sec

Epoch 9 Batch 0 Loss 0.8039
Time taken for 1 epoch 325.38785219192505 sec

Epoch 10 Batch 0 Loss 0.6268
Time taken for 1 epoch 325.6905493736267 sec

Epoch 11 Batch 0 Loss 0.4681
Time taken for 1 epoch 325.11986088752747 sec

Epoch 12 Batch 0 Loss 0.3792
Time taken for 1 epoch 325.97212433815 sec

Epoch 13 Batch 0 Loss 0.3588
Time taken for 1 epoch 325.16398215293884 sec

Epoch 14 Batch 0 Loss 0.3038
Ti

In [ ]:
checkpoint.restore(tf.train.latest_checkpoint(checkpoint_dir))

In [ ]:
def evaluate(inputs, encoder, decoder, inp_lang, targ_lang, max_length_inp, max_length_targ):

    attention_plot = np.zeros((max_length_targ, max_length_inp))
    sentence = ''
    for i in inputs[0]:
        if i == 0:
            break
        sentence = sentence + inp_lang.idx2word[i] + ' '
    sentence = sentence[:-1]

    inputs = tf.convert_to_tensor(inputs)

    result = ''

    hidden = [tf.zeros((1, units))]
    enc_out, enc_hidden = encoder(inputs, hidden)

    dec_hidden = enc_hidden
    dec_input = tf.expand_dims([targ_lang.word2idx['<start>']], 0)

    for t in range(max_length_targ):
        predictions, dec_hidden, attention_weights = decoder(dec_input, dec_hidden, enc_out)

        # storing the attention weights to plot later on
        attention_weights = tf.reshape(attention_weights, (-1, ))
        attention_plot[t] = attention_weights.numpy()

        predicted_id = tf.argmax(predictions[0]).numpy()

        result += targ_lang.idx2word[predicted_id] + ' '

        if targ_lang.idx2word[predicted_id] == '<end>':
            return result, sentence, attention_plot

        # the predicted ID is fed back into the model
        dec_input = tf.expand_dims([predicted_id], 0)

    return result, sentence, attention_plot

In [ ]:
def predict_random_val_sentence():
    actual_sent = ''
    k = np.random.randint(len(input_tensor_val))
    random_input = input_tensor_val[k]
    random_output = target_tensor_val[k]
    random_input = np.expand_dims(random_input,0)
    result, sentence, attention_plot = evaluate(random_input, encoder, decoder, inp_lang, targ_lang, max_length_inp, max_length_targ)
    print('Input: {}'.format(sentence[8:-6]))

    for i in random_output:
        if i == 0:
            break
        actual_sent = actual_sent + targ_lang.idx2word[i] + ' '
    actual_sent = actual_sent[8:-7]
    print('Actual translation: {}'.format(actual_sent))
    print('Predicted translation: {}'.format(result[:-6]))
    BLEUscore = nltk.translate.bleu_score.sentence_bleu([result], sentence, weights = (0.5, 0.5))
    print(BLEUscore)

    attention_plot = attention_plot[:len(result.split(' '))-2, 1:len(sentence.split(' '))-1]
    sentence, result = sentence.split(' '), result.split(' ')
    sentence = sentence[1:-1]
    result = result[:-2]

    # use plotly to generate the heat map
    trace = go.Heatmap(z = attention_plot, x = sentence, y = result, colorscale='blues')
    data=[trace]
    iplot(data)


In [ ]:
!pip install nltk
import nltk

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
predict_random_val_sentence()

Input: there are at least six tanzanians of indian origin who are now serving as members of parliament in tanzania
Actual translation: ಭಾರತೀಯ ಮೂಲದ ಕನಿಷ್ಠ ಆರು ತಾಂಜಾನಿಯಾದವರು ಈಗ ತಾಂಜಾನಿಯಾದಲ್ಲಿ ಸಂಸತ್ತಿನ ಸದಸ್ಯರಾಗಿ ಸೇವೆ ಸಲ್ಲಿಸುತ್ತಿದ್ದಾರೆ
Predicted translation: ಭಾರತೀಯ ಮೂಲದ ಕನಿಷ್ಠ ಆರು ತಾಂಜಾನಿಯಾದವರು ಈಗ ತಾಂಜಾನಿಯಾದಲ್ಲಿ ಸಂಸತ್ತಿನ ಸದಸ್ಯರಾಗಿ ಸೇವೆ ಸಲ್ಲಿಸುತ್ತಿದ್ದಾರೆ 
0.08381404052084444


In [ ]:
predict_random_val_sentence()

Input: slavery does not merely mean a legalised form of subjection
Actual translation: ಗುಲಾಮಗಿರಿಯು ಕೇವಲ ಅಧೀನತೆಯ ಕಾನೂನುಬದ್ಧ ರೂಪವಲ್ಲ
Predicted translation: ಗುಲಾಮಗಿರಿಯು ಕೇವಲ ಅಧೀನತೆಯ ಕಾನೂನುಬದ್ಧ ರೂಪವಲ್ಲ 
0.11205836704847512


In [ ]:
predict_random_val_sentence()

Input: the gujarati flair for business is well known
Actual translation: ವ್ಯಾಪಾರಕ್ಕಾಗಿ ಗುಜರಾತಿನ ಕೌಶಲ್ಯವು ಪ್ರಸಿದ್ಧವಾಗಿದೆ
Predicted translation: ವ್ಯಾಪಾರಕ್ಕಾಗಿ ಗುಜರಾತಿನ ಕೌಶಲ್ಯವು ಪ್ರಸಿದ್ಧವಾಗಿದೆ 
0.13241452366835288


In [ ]:
predict_random_val_sentence()

Input: life becomes more securemore fraught with actionricher in achievement and experience
Actual translation: ಜೀವನವು ಹೆಚ್ಚು ಸುರಕ್ಷಿತವಾಗುತ್ತದೆ ಹೆಚ್ಚು ಕ್ರಿಯೆಯಿಂದ ತುಂಬಿರುತ್ತದೆ ಸಾಧನೆ ಮತ್ತು ಅನುಭವದಲ್ಲಿ ಶ್ರೀಮಂತವಾಗುತ್ತದೆ
Predicted translation: ಜೀವನವು ಹೆಚ್ಚು ಸುರಕ್ಷಿತವಾಗುತ್ತದೆ ಹೆಚ್ಚು ಕ್ರಿಯೆಯಿಂದ ತುಂಬಿರುತ್ತದೆ ಸಾಧನೆ ಮತ್ತು ಅನುಭವದಲ್ಲಿ ಶ್ರೀಮಂತವಾಗುತ್ತದೆ 
0.08982337554486226


In [ ]:
predict_random_val_sentence()

Input: superstitions is our great enemy but bigotry is worse
Actual translation: ಮೂಢನಂಬಿಕೆಗಳು ನಮ್ಮ ದೊಡ್ಡ ಶತ್ರು ಆದರೆ ಮತಾಂಧತೆ ಕೆಟ್ಟದಾಗಿದೆ
Predicted translation: ಮೂಢನಂಬಿಕೆಗಳು ನಮ್ಮ ದೊಡ್ಡ ಶತ್ರು ಆದರೆ ಮತಾಂಧತೆ ಕೆಟ್ಟದಾಗಿದೆ 
0.13281218822295593


In [ ]:
predict_random_val_sentence()

Input: agriculture has a very big role in it
Actual translation: ಅದರಲ್ಲಿ ಕೃಷಿಯ ಪಾತ್ರ ಬಹಳ ದೊಡ್ಡದು
Predicted translation: ಅದರಲ್ಲಿ ಕೃಷಿಯ ಪಾತ್ರ ಬಹಳ ದೊಡ್ಡದು 
0.16087993330796874


In [ ]:
predict_random_val_sentence()

Input: great things come afterwards
Actual translation: ದೊಡ್ಡ ವಿಷಯಗಳು ನಂತರ ಬರುತ್ತವೆ
Predicted translation: ದೊಡ್ಡ ವಿಷಯಗಳು ನಂತರ ಬರುತ್ತವೆ 
0.18666334823663935


In [ ]:
predict_random_val_sentence()

Input: a great man is different from an eminent one in that he is ready to be the servant of the society
Actual translation: ಒಬ್ಬ ಮಹಾನ್ ವ್ಯಕ್ತಿ ಶ್ರೇಷ್ಠರಿಗಿಂತ ಭಿನ್ನವಾಗಿರುತ್ತಾನೆ ಅವನು ಸಮಾಜದ ಸೇವಕನಾಗಲು ಸಿದ್ಧನಾಗಿದ್ದಾನೆ
Predicted translation: ಒಬ್ಬ ಮಹಾನ್ ವ್ಯಕ್ತಿ ಶ್ರೇಷ್ಠರಿಗಿಂತ ಭಿನ್ನವಾಗಿರುತ್ತಾನೆ ಅವನು ಸಮಾಜದ ಸೇವಕನಾಗಲು ಸಿದ್ಧನಾಗಿದ್ದಾನೆ 
0.08585457105482136


In [ ]:
predict_random_val_sentence()

Input: untouchability has ruined the untouchables the hindus and ultimately the nation as well
Actual translation: ಅಸ್ಪೃಶ್ಯತೆಯು ಅಸ್ಪೃಶ್ಯರನ್ನು ಹಿಂದೂಗಳನ್ನು ಮತ್ತು ಅಂತಿಮವಾಗಿ ರಾಷ್ಟ್ರವನ್ನು ಹಾಳುಮಾಡಿದೆ
Predicted translation: ಅಸ್ಪೃಶ್ಯತೆಯು ಅಸ್ಪೃಶ್ಯರನ್ನು ಹಿಂದೂಗಳನ್ನು ಮತ್ತು ಅಂತಿಮವಾಗಿ ರಾಷ್ಟ್ರವನ್ನು ಹಾಳುಮಾಡಿದೆ 
0.08787930517037401


In [ ]:
predict_random_val_sentence()

Input: it certainly follows that whatever we wish to be in future can be produced by our present actionsso we have to know how to act
Actual translation: ಭವಿಷ್ಯದಲ್ಲಿ ನಾವು ಏನಾಗಬೇಕೆಂದು ಬಯಸುತ್ತೇವೆಯೋ ಅದನ್ನು ನಮ್ಮ ಪ್ರಸ್ತುತ ಕ್ರಿಯೆಗಳಿಂದ ಉತ್ಪಾದಿಸಬಹುದು ಎಂದು ಅದು ಖಂಡಿತವಾಗಿಯೂ ಅನುಸರಿಸುತ್ತದೆ ಆದ್ದರಿಂದ ನಾವು ಹೇಗೆ ವರ್ತಿಸಬೇಕು ಎಂದು ತಿಳಿದಿರಬೇಕು
Predicted translation: ಭವಿಷ್ಯದಲ್ಲಿ ನಾವು ಏನಾಗಬೇಕೆಂದು ಬಯಸುತ್ತೇವೆಯೋ ಅದನ್ನು ನಮ್ಮ ಪ್ರಸ್ತುತ ಕ್ರಿಯೆಗಳಿಂದ ಉತ್ಪಾದಿಸಬಹುದು ಎಂದು ಅದು ಖಂಡಿತವಾಗಿಯೂ ಅನುಸರಿಸುತ್ತದೆ ಆದ್ದರಿಂದ ನಾವು ಹೇಗೆ ವರ್ತಿಸಬೇಕು ಎಂದು ತಿಳಿದಿರಬೇಕು 
0.06740566291333827


In [ ]:
predict_random_val_sentence()

Input: india desires a peaceful and stable environment for its development
Actual translation: ಭಾರತವು ತನ್ನ ಅಭಿವೃದ್ಧಿಗಾಗಿ ಶಾಂತಿಯುತ ಮತ್ತು ಸ್ಥಿರ ವಾತಾವರಣವನ್ನು ಬಯಸುತ್ತದೆ
Predicted translation: ಭಾರತವು ತನ್ನ ಅಭಿವೃದ್ಧಿಗಾಗಿ ಶಾಂತಿಯುತ ಮತ್ತು ಸ್ಥಿರ ವಾತಾವರಣವನ್ನು ಬಯಸುತ್ತದೆ 
0.11385500851066219


In [ ]:
predict_random_val_sentence()

Input: the greatest help to spiritual life is meditation in meditation we divest ourselves of all material condition and feel our divine nature
Actual translation: ಆಧ್ಯಾತ್ಮಿಕ ಜೀವನಕ್ಕೆ ದೊಡ್ಡ ಸಹಾಯವೆಂದರೆ ಧ್ಯಾನ ಧ್ಯಾನದಲ್ಲಿ ನಾವು ಎಲ್ಲಾ ಭೌತಿಕ ಸ್ಥಿತಿಗಳಿಂದ ದೂರವಿರುತ್ತೇವೆ ಮತ್ತು ನಮ್ಮ ದೈವಿಕ ಸ್ವಭಾವವನ್ನು ಅನುಭವಿಸುತ್ತೇವೆ
Predicted translation: ಆಧ್ಯಾತ್ಮಿಕ ಜೀವನಕ್ಕೆ ದೊಡ್ಡ ಸಹಾಯವೆಂದರೆ ಧ್ಯಾನ ಧ್ಯಾನದಲ್ಲಿ ನಾವು ಎಲ್ಲಾ ಭೌತಿಕ ಸ್ಥಿತಿಗಳಿಂದ ದೂರವಿರುತ್ತೇವೆ ಮತ್ತು ನಮ್ಮ ದೈವಿಕ ಸ್ವಭಾವವನ್ನು ಅನುಭವಿಸುತ್ತೇವೆ 
0.07685076621905845


In [ ]:
predict_random_val_sentence()

Input: this charter also contains certain requirements from taxpayers
Actual translation: ಈ ಚಾರ್ಟರ್ ತೆರಿಗೆದಾರರಿಂದ ಕೆಲವು ಅವಶ್ಯಕತೆಗಳನ್ನು ಸಹ ಒಳಗೊಂಡಿದೆ
Predicted translation: ಈ ಚಾರ್ಟರ್ ತೆರಿಗೆದಾರರಿಂದ ಕೆಲವು ಅವಶ್ಯಕತೆಗಳನ್ನು ಸಹ ಒಳಗೊಂಡಿದೆ 
0.11697953037312032


In [ ]:
predict_random_val_sentence()

Input: i am sure a very large portion of the vils and miseries that have would have vanished
Actual translation: ಮಾಯವಾಗುತ್ತಿದ್ದ ವಿಲ್‌ಗಳು ಮತ್ತು ದುಃಖಗಳ ಬಹುಪಾಲು ಭಾಗವು ಕಣ್ಮರೆಯಾಗುತ್ತದೆ ಎಂದು ನನಗೆ ಖಾತ್ರಿಯಿದೆ
Predicted translation: ಮಾಯವಾಗುತ್ತಿದ್ದ ವಿಲ್‌ಗಳು ಮತ್ತು ದುಃಖಗಳ ಬಹುಪಾಲು ಭಾಗವು ಕಣ್ಮರೆಯಾಗುತ್ತದೆ ಎಂದು ನನಗೆ ಖಾತ್ರಿಯಿದೆ 
0.09947294626039879


In [ ]:
predict_random_val_sentence()

Input: it is not enough to be electors only it is necessary to be lawmakers
Actual translation: ಕೇವಲ ಮತದಾರರಾದರೆ ಸಾಲದು ಕಾನೂನು ರೂಪಿಸುವವರಾಗುವುದು ಅವಶ್ಯಕ
Predicted translation: ಕೇವಲ ಮತದಾರರಾದರೆ ಸಾಲದು ಕಾನೂನು ರೂಪಿಸುವವರಾಗುವುದು ಅವಶ್ಯಕ 
0.10411584125907072


In [ ]:
predict_random_val_sentence()

Input: i will drink the ocean
Actual translation: ನಾನು ಸಾಗರವನ್ನು ಕುಡಿಯುತ್ತೇನೆ
Predicted translation: ನಾನು ಸಾಗರವನ್ನು ಕುಡಿಯುತ್ತೇನೆ 
0.20701966780270625


In [ ]:
predict_random_val_sentence()

Input: god is always god but the views which people and nations may take of him vary no higher view is known than that of love
Actual translation: ದೇವರು ಯಾವಾಗಲೂ ದೇವರು ಆದರೆ ಜನರು ಮತ್ತು ರಾಷ್ಟ್ರಗಳು ಅವನ ಬಗ್ಗೆ ತೆಗೆದುಕೊಳ್ಳುವ ದೃಷ್ಟಿಕೋನಗಳು ಬದಲಾಗುತ್ತವೆ ಪ್ರೀತಿಗಿಂತ ಹೆಚ್ಚಿನ ದೃಷ್ಟಿಕೋನವು ತಿಳಿದಿಲ್ಲ
Predicted translation: ದೇವರು ಯಾವಾಗಲೂ ದೇವರು ಆದರೆ ಜನರು ಮತ್ತು ರಾಷ್ಟ್ರಗಳು ಅವನ ಬಗ್ಗೆ ತೆಗೆದುಕೊಳ್ಳುವ ದೃಷ್ಟಿಕೋನಗಳು ಬದಲಾಗುತ್ತವೆ ಪ್ರೀತಿಗಿಂತ ಹೆಚ್ಚಿನ ದೃಷ್ಟಿಕೋನವು ತಿಳಿದಿಲ್ಲ 
0.08043044269038903


In [ ]:
predict_random_val_sentence()

Input: it is not enough to be electors only it is necessary to be lawmakers
Actual translation: ಕೇವಲ ಮತದಾರರಾದರೆ ಸಾಲದು ಕಾನೂನು ರೂಪಿಸುವವರಾಗುವುದು ಅವಶ್ಯಕ
Predicted translation: ಕೇವಲ ಮತದಾರರಾದರೆ ಸಾಲದು ಕಾನೂನು ರೂಪಿಸುವವರಾಗುವುದು ಅವಶ್ಯಕ 
0.10411584125907072


In [ ]:
predict_random_val_sentence()

Input: know that you are the creator of your own destiny
Actual translation: ನಿಮ್ಮ ಸ್ವಂತ ಹಣೆಬರಹದ ಸೃಷ್ಟಿಕರ್ತ ನೀವೇ ಎಂದು ತಿಳಿಯಿರಿ
Predicted translation: ನಿಮ್ಮ ಸ್ವಂತ ಹಣೆಬರಹದ ಸೃಷ್ಟಿಕರ್ತ ನೀವೇ ಎಂದು ತಿಳಿಯಿರಿ 
0.14131269593987003


In [ ]:
predict_random_val_sentence()

Input: the st century is an era of knowledge
Actual translation: 21 ನೇ ಶತಮಾನವು ಜ್ಞಾನದ ಯುಗವಾಗಿದೆ
Predicted translation: 21 ನೇ ಶತಮಾನವು ಜ್ಞಾನದ ಯುಗವಾಗಿದೆ 
0.16087993330796874


In [ ]:
predict_random_val_sentence()

Input: agriculture has a very big role in it
Actual translation: ಅದರಲ್ಲಿ ಕೃಷಿಯ ಪಾತ್ರ ಬಹಳ ದೊಡ್ಡದು
Predicted translation: ಅದರಲ್ಲಿ ಕೃಷಿಯ ಪಾತ್ರ ಬಹಳ ದೊಡ್ಡದು 
0.16087993330796874


In [ ]:
predict_random_val_sentence()

Input: we are responsible whatever we wish ourselves to be
Actual translation: ನಾವು ಏನಾಗಬೇಕೆಂದು ಬಯಸುತ್ತೇವೆಯೋ ಅದಕ್ಕೆ ನಾವು ಜವಾಬ್ದಾರರು
Predicted translation: ನಾವು ಏನಾಗಬೇಕೆಂದು ಬಯಸುತ್ತೇವೆಯೋ ಅದಕ್ಕೆ ನಾವು ಜವಾಬ್ದಾರರು 
0.13155870289605437


In [ ]:
predict_random_val_sentence()

Input: sea waves give the challenges
Actual translation: ಸಮುದ್ರದ ಅಲೆಗಳು ಸವಾಲುಗಳನ್ನು ನೀಡುತ್ತವೆ
Predicted translation: ಸಮುದ್ರದ ಅಲೆಗಳು ಸವಾಲುಗಳನ್ನು ನೀಡುತ್ತವೆ 
0.18227065414412227


In [ ]:
predict_random_val_sentence()

Input: if you study carefully you will see that buddhism is based on reason
Actual translation: ನೀವು ಎಚ್ಚರಿಕೆಯಿಂದ ಅಧ್ಯಯನ ಮಾಡಿದರೆ ಬೌದ್ಧಧರ್ಮವು ಕಾರಣವನ್ನು ಆಧರಿಸಿದೆ ಎಂದು ನೀವು ನೋಡುತ್ತೀರಿ
Predicted translation: ನೀವು ಎಚ್ಚರಿಕೆಯಿಂದ ಅಧ್ಯಯನ ಮಾಡಿದರೆ ಬೌದ್ಧಧರ್ಮವು ಕಾರಣವನ್ನು ಆಧರಿಸಿದೆ ಎಂದು ನೀವು ನೋಡುತ್ತೀರಿ 
0.10772579637991721


In [ ]:
predict_random_val_sentence()

Input: this charter also contains certain requirements from taxpayers
Actual translation: ಈ ಚಾರ್ಟರ್ ತೆರಿಗೆದಾರರಿಂದ ಕೆಲವು ಅವಶ್ಯಕತೆಗಳನ್ನು ಸಹ ಒಳಗೊಂಡಿದೆ
Predicted translation: ಈ ಚಾರ್ಟರ್ ತೆರಿಗೆದಾರರಿಂದ ಕೆಲವು ಅವಶ್ಯಕತೆಗಳನ್ನು ಸಹ ಒಳಗೊಂಡಿದೆ 
0.11697953037312032


In [ ]:
predict_random_val_sentence()

Input: arise awake sleep no more within each of you there is the power to remove all wants and allmiseries believe this and that power will be manifested
Actual translation: ಎದ್ದೇಳು ಎಚ್ಚರಗೊಳ್ಳು ಇನ್ನು ನಿದ್ರೆ ಮಾಡಬೇಡ ನಿಮ್ಮಲ್ಲಿ ಪ್ರತಿಯೊಬ್ಬರೊಳಗೆ ಎಲ್ಲಾ ಆಸೆಗಳನ್ನು ಮತ್ತು ಎಲ್ಲಾ ಸರಣಿಗಳನ್ನು ತೆಗೆದುಹಾಕುವ ಶಕ್ತಿ ಇದೆ ಇದನ್ನು ನಂಬಿರಿ ಮತ್ತು ಶಕ್ತಿಯು ಪ್ರಕಟವಾಗುತ್ತದೆ
Predicted translation: ಎದ್ದೇಳು ಎಚ್ಚರಗೊಳ್ಳು ಇನ್ನು ನಿದ್ರೆ ಮಾಡಬೇಡ ನಿಮ್ಮಲ್ಲಿ ಪ್ರತಿಯೊಬ್ಬರೊಳಗೆ ಎಲ್ಲಾ ಆಸೆಗಳನ್ನು ಮತ್ತು ಎಲ್ಲಾ ಸರಣಿಗಳನ್ನು ತೆಗೆದುಹಾಕುವ ಶಕ್ತಿ ಇದೆ ಇದನ್ನು ನಂಬಿರಿ ಮತ್ತು ಶಕ್ತಿಯು ಪ್ರಕಟವಾಗುತ್ತದೆ 
0.07085561917642393


In [ ]:
predict_random_val_sentence()

Input: yoga gives a guarantee not only for fitness but also for wellness
Actual translation: ಯೋಗವು ಫಿಟ್‌ನೆಸ್‌ಗೆ ಮಾತ್ರವಲ್ಲದೆ ಕ್ಷೇಮಕ್ಕೂ ಗ್ಯಾರಂಟಿ ನೀಡುತ್ತದೆ
Predicted translation: ಯೋಗವು ಫಿಟ್‌ನೆಸ್‌ಗೆ ಮಾತ್ರವಲ್ಲದೆ ಕ್ಷೇಮಕ್ಕೂ ಗ್ಯಾರಂಟಿ ನೀಡುತ್ತದೆ 
0.108094949066235


In [ ]:
predict_random_val_sentence()

Input: he smilingly said that it was from by birth
Actual translation: ಅದು ಹುಟ್ಟಿನಿಂದ ಬಂದದ್ದು ಎಂದು ನಗುತ್ತಾ ಹೇಳಿದರು
Predicted translation: ಅದು ಹುಟ್ಟಿನಿಂದ ಬಂದದ್ದು ಎಂದು ನಗುತ್ತಾ ಹೇಳಿದರು 
0.15018785229652765


In [ ]:
predict_random_val_sentence()

Input: overcome problems through partnership
Actual translation: ಪಾಲುದಾರಿಕೆಯ ಮೂಲಕ ಸಮಸ್ಯೆಗಳನ್ನು ನಿವಾರಿಸಿ
Predicted translation: ಪಾಲುದಾರಿಕೆಯ ಮೂಲಕ ಸಮಸ್ಯೆಗಳನ್ನು ನಿವಾರಿಸಿ 
0.15339299776947407


In [ ]:
predict_random_val_sentence()

Input: it is only by doing good to others that one attains to ones own good
Actual translation: ಇತರರಿಗೆ ಒಳ್ಳೆಯದನ್ನು ಮಾಡುವುದರಿಂದ ಮಾತ್ರ ಒಬ್ಬನು ತನ್ನ ಒಳಿತನ್ನು ಸಾಧಿಸುತ್ತಾನೆ
Predicted translation: ಇತರರಿಗೆ ಒಳ್ಳೆಯದನ್ನು ಮಾಡುವುದರಿಂದ ಮಾತ್ರ ಒಬ್ಬನು ತನ್ನ ಒಳಿತನ್ನು ಸಾಧಿಸುತ್ತಾನೆ 
0.11245796163277197


In [ ]:
predict_random_val_sentence()

Input: agriculture has a very big role in it
Actual translation: ಅದರಲ್ಲಿ ಕೃಷಿಯ ಪಾತ್ರ ಬಹಳ ದೊಡ್ಡದು
Predicted translation: ಅದರಲ್ಲಿ ಕೃಷಿಯ ಪಾತ್ರ ಬಹಳ ದೊಡ್ಡದು 
0.16087993330796874


In [ ]:
predict_random_val_sentence()

Input: historically communities from western india especially gujarat and the eastern coast of africa have settled in each other’s lands
Actual translation: ಐತಿಹಾಸಿಕವಾಗಿ ಪಶ್ಚಿಮ ಭಾರತ ವಿಶೇಷವಾಗಿ ಗುಜರಾತ್ ಮತ್ತು ಆಫ್ರಿಕಾದ ಪೂರ್ವ ಕರಾವಳಿಯ ಸಮುದಾಯಗಳು ಪರಸ್ಪರರ ಭೂಮಿಯಲ್ಲಿ ನೆಲೆಸಿದ್ದಾರೆ
Predicted translation: ಐತಿಹಾಸಿಕವಾಗಿ ಪಶ್ಚಿಮ ಭಾರತ ವಿಶೇಷವಾಗಿ ಗುಜರಾತ್ ಮತ್ತು ಆಫ್ರಿಕಾದ ಪೂರ್ವ ಕರಾವಳಿಯ ಸಮುದಾಯಗಳು ಪರಸ್ಪರರ ಭೂಮಿಯಲ್ಲಿ ನೆಲೆಸಿದ್ದಾರೆ 
0.07492732616111096


In [ ]:
predict_random_val_sentence()

Input: freedom of mind is the proof of ones existence
Actual translation: ಮನಸ್ಸಿನ ಸ್ವಾತಂತ್ರ್ಯವು ಒಬ್ಬರ ಅಸ್ತಿತ್ವದ ಪುರಾವೆಯಾಗಿದೆ
Predicted translation: ಮನಸ್ಸಿನ ಸ್ವಾತಂತ್ರ್ಯವು ಒಬ್ಬರ ಅಸ್ತಿತ್ವದ ಪುರಾವೆಯಾಗಿದೆ 
0.1365432817703243


In [ ]:
predict_random_val_sentence()

Input: our nation is rich and our people can be rich
Actual translation: ನಮ್ಮ ದೇಶ ಶ್ರೀಮಂತವಾಗಿದೆ ಮತ್ತು ನಮ್ಮ ಜನರು ಶ್ರೀಮಂತರಾಗಬಹುದು
Predicted translation: ನಮ್ಮ ದೇಶ ಶ್ರೀಮಂತವಾಗಿದೆ ಮತ್ತು ನಮ್ಮ ಜನರು ಶ್ರೀಮಂತರಾಗಬಹುದು 
0.14594373343793487


In [ ]:
predict_random_val_sentence()

Input: all will work according to their respective obligations
Actual translation: ಎಲ್ಲರೂ ತಮ್ಮ ತಮ್ಮ ಜವಾಬ್ದಾರಿಗಳ ಪ್ರಕಾರ ಕೆಲಸ ಮಾಡುತ್ತಾರೆ
Predicted translation: ಎಲ್ಲರೂ ತಮ್ಮ ತಮ್ಮ ಜವಾಬ್ದಾರಿಗಳ ಪ್ರಕಾರ ಕೆಲಸ ಮಾಡುತ್ತಾರೆ 
0.1289342503786954


In [ ]:
predict_random_val_sentence()

Input: the taste of anything can be changed
Actual translation: ಯಾವುದೇ ವಸ್ತುವಿನ ರುಚಿಯನ್ನು ಬದಲಾಯಿಸಬಹುದು
Predicted translation: ಯಾವುದೇ ವಸ್ತುವಿನ ರುಚಿಯನ್ನು ಬದಲಾಯಿಸಬಹುದು 
0.15649215928719032


In [ ]:
predict_random_val_sentence()

Input: india’s partnership with africa is based on a model of cooperation which is responsive to the needs of african countries
Actual translation: ಆಫ್ರಿಕಾದೊಂದಿಗಿನ ಭಾರತದ ಪಾಲುದಾರಿಕೆಯು ಆಫ್ರಿಕನ್ ದೇಶಗಳ ಅಗತ್ಯಗಳಿಗೆ ಸ್ಪಂದಿಸುವ ಸಹಕಾರದ ಮಾದರಿಯನ್ನು ಆಧರಿಸಿದೆ
Predicted translation: ಆಫ್ರಿಕಾದೊಂದಿಗಿನ ಭಾರತದ ಪಾಲುದಾರಿಕೆಯು ಆಫ್ರಿಕನ್ ದೇಶಗಳ ಅಗತ್ಯಗಳಿಗೆ ಸ್ಪಂದಿಸುವ ಸಹಕಾರದ ಮಾದರಿಯನ್ನು ಆಧರಿಸಿದೆ 
0.0733934658565345
